In [ ]:
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
url = "https://www.imdb.com/search/title/?num_votes=2500,"

driver = webdriver.Chrome()
driver.get(url)

driver.implicitly_wait(5)

# Find button with text "accept" to get rid of the cookie banner
cookie_button = driver.find_element(By.XPATH, "//button[text()='Accept']")
cookie_button.click()

# Store the data in a set to prevent duplicates
data = set()

In [ ]:
while (len(data) < 30000):

    try:
        # Get all movies
        movies = driver.find_elements(By.CLASS_NAME, "ipc-metadata-list-summary-item")

        # Extract the urls and description
        for movie in movies:
            url = movie.find_element(By.TAG_NAME, "a").get_attribute("href")
            description = movie.find_element(By.CLASS_NAME, "ipc-html-content-inner-div").text 
            data.add((url, description))
        
        # Convert urls to list and store them
        data_list = list(data)
        with open("urls.json", "w") as f:
            json.dump(data_list, f, ensure_ascii=False, indent=4)

        # Wait until the "50 mehr" button is visible and clickable
        button = WebDriverWait(driver, 15).until(
            EC.element_to_be_clickable((By.XPATH, "//button[.//span[@class='ipc-see-more__text' and text()='50 mehr']]"))
        )

        # Scroll the button into view using JavaScript
        driver.execute_script("arguments[0].scrollIntoView(true);", button)
        
        # Click the button using JavaScript to bypass overlay issues, likley caused by anti-scrape-mechanisms
        driver.execute_script("arguments[0].click();", button)

        print(len(data))

    except Exception as e:
        print(e)
        continue

driver.quit()